<img align='center' style='max-width: 1100px' src='banner.jpg'>

<img align="right" style="max-width: 250px; height: auto" src="sfao_logo.png">

## Agentic Accounting Journal Entry Testing

Code Lab 001 - World Continuous Auditing & Reporting Symposium 

Our lab environment is based on `Jupyter Notebooks` (https://jupyter.org/), which provide an interactive platform for performing a variety of statistical evaluations and data analyses. In this lab, we will explore how to apply **Artificial Intelligence (AI)**, specifically using the `crewAI` framework, to perform automated auditing tasks. The objective is to leverage multiple AI agents that can collaboratively analyze accounting journal entries, simulating the work of human auditors. This approach is based on the concept of **Agentic Auditing**, where specialized AI agents are assigned distinct tasks, such as anomaly detection, compliance verification, and approval flow analysis. Each agent operates autonomously but works as part of a team, ensuring that audit processes are thorough and efficient.

<img align="center" style="max-width: 950px" src="./agent.png">

**Figure 1: Artificial Intelligence Agents**

The application of AI in auditing has evolved rapidly, with foundational work in automation and AI-driven decision-making influencing the future of audit practices. Researchers and practitioners alike are now exploring how to deploy these agents in real-world audit tasks, such as detecting irregularities in financial records, improving the accuracy of audits, and reducing manual effort. In this lab, we will implement and run multiple agents using the `crewAI` framework to collaboratively audit journal entries. By the end of the lab, you will understand how to set up AI agents, assign them specific audit tasks, and have them work together to identify anomalies in accounting data.

The figure below provides a high-level overview of the **agentic journal entry testing process** we aim to establish in this lab.

<img align="center" style="max-width: 900px" src="./audit_process.png">

**Figure 2: Agentic Journal Entry Testing Process**

Using the `crewAI` framework (https://www.crewai.com/), we will implement and deploy a team of AI agents to audit accounting journal entries. Each agent will perform specific tasks such as anomaly detection, compliance checks, or approval flow analysis. After setting up the agents, we will have them work together to analyze the journal entries and identify any irregularities.

As always, pls. don't hesitate to ask all your questions either during the lab, post them in our CANVAS (StudyNet) forum (https://learning.unisg.ch), or send us an email (using the course email).

## 1. Lab Objectives:

After today's lab, you should be able to:

> 1. **Understand Agentic Auditing:** Grasp the basics of using AI agents for auditing tasks.
> 2. **Implement AI Agents with crewAI:** Set up and configure agents to perform audit tasks.
> 3. **Apply Agents for Journal Entry Testing:** Use agents to detect anomalies in journal entries.
> 4. **Evaluate Agent Performance:** Analyze the performance of agents using audit-relevant metrics.
> 5. **Report Audit Outcomes:** Visualize and summarize audit results.


## 2. Setup of the Jupyter Notebook Environment

We need to import several Python libraries that facilitate data analysis and visualization. We will primarily use `crewAI`, `SqlAlchemy`, `Langchain`, and a few utility libraries throughout this lab:

In [ ]:
# import text processing, database and utility libraries
import os
import pandas as pd
import tabulate as tb
from datetime import datetime
from IPython.display import Markdown, display
from textwrap import dedent
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Date, insert

Import the `crewAI`agentic framework library:

In [ ]:
# import crewai library and tools
from crewai import Crew, Agent, Process, Task
from crewai_tools import BaseTool

Import the `langchain` LLM chaining library:

In [ ]:
# import langchain
from langchain_openai import ChatOpenAI

## 3. Setup of the Local Large Language Model (LLM)

In this section, we set up the necessary environment variables and initialize the access to a local **Large Language Model (LLM)**, which the agents will use to perform different audit tasks, such as reviewing organizational guidelines. In this step, we set up a workaround by setting a few dummy environment variables required for the **Langchain** `ChatOpenAI` interface. The environment variables would typically store API keys, but since we're working with a local LLM, these are placeholders.

In [ ]:
# set dummy environment variables
os.environ['OPENAI_API_KEY'] = 'Na'  # Masked API key

Next, we will use the ChatOpenAI` interface to connect to a to a locally hosted **LLaMA3** model. The model will be used by the agents to perform natural language tasks during the audit process: 

In [ ]:
# initialize the local llama 3.0 llm by meta.ai
llm = ChatOpenAI(model='crewai-llama3', base_url='http://localhost:11434/v1')

Before we dive into the more formal auditing tasks, let’s have some fun testing the local **LLaMA3** model. We’ll simulate a scenario where the AI is asked to audit an expense report for a quirky employee who might be trying to sneak in some personal purchases. This test will ensure our model is responding correctly while adding a bit of humor to the process:

In [ ]:
# define audit prompt 
prompt = """
Imagine you're an auditor reviewing an expense report from an employee who claims all expenses are legitimate business costs. 
Here's a list of items they've submitted:

1. Office supplies - $100
2. Coffee machine - $250
3. Unicorn onesie - $45
4. 10-pound gummy bear - $30
5. "Business trip" to Disneyland - $2000
6. Sushi for team meeting - $150

Please identify which items are legitimate business expenses and which might require further investigation, with a bit of humor in your response.
"""

Next, the local **LLaMA3** model is tasked with determining which items in the expense report are legitimate and which require further investigation. The model should provide a humorous yet practical analysis of the expenses:

In [ ]:
# test the model with this fun audit prompt
print(llm.invoke(prompt).content)

## 4. Establishing the Senior Auditor Agent

In this section, we will create the first agent of the audit crew, the **Senior Auditor**. The agent is skilled in extracting the key points from complex guidelines and turning them into actionable insights. Let's walk through the steps to create and assign tasks to this agent.

<img align='center' style="max-width: 850px" src='./senior_auditor_agent.png'>

**Figure 3: Senior Auditor Agent - Tools and Capabilities**

The **Senior Auditor** agent will need a tool to read the organizational guidelines. We define a class called `ReadGuidelines` that reads a text file containing the guidelines and returns its content. This will allow the agent to access and process the guidelines.

In [ ]:
# define the ReadGuidelines tool
class ReadGuidelines(BaseTool):

    name: str = 'Read Guidelines'
    description: str = 'Read organisational guidelines and return the guideline content.'
    result_as_answer: bool = True

    # define database query
    def _run(self, query: str) -> str:
        """Read a text file from the given file path and query its content."""
        try:
            file_path='./guideline.txt'
            with open(file_path, 'r') as file:
                content = file.read()
            return str(content)

        except Exception as exception:
            return str(exception)

In [ ]:
# instantiate the ReadGuidelines tool
rga = ReadGuidelines()

Next, we define the **Senior Auditor** agent. This agent is designed to read the organizational guidelines using the `ReadGuidelines` tool and summarize them in a clear and concise way, particularly focusing on relevant points for an IT auditor.

The Senior Auditor agent has the following characteristics:
- **Goal:** To analyze comprehensive and complex organizational guidelines and extract their main points.
- **Backstory:** A skilled auditor known for turning complex guidelines into actionable insights.

In [ ]:
# define the Senior Auditor agent
senior_auditor = Agent(
    role='Senior Auditor',
    goal=dedent("""\
        Analyze comprehensive and complex organizational policy and extract their main points.
    """),
    backstory=dedent("""\
        You are a skilled Senior Auditor, celebrated for your ability to analyze organizational guidelines. 
        Your analysis reports distill complex guidelines into clear, concise, and actionable insights. 
        You excel in transforming complex regulations into understandable summaries that can be easily utilized by other audit team members.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm, # the agent will use the LLaMA3 model
    tools=[rga] # the agent will use the ReadGuidelines tool
)

Now that the **Senior Auditor** is defined, we assign a task for the agent. The task is to read and summarize the guideline in a concise manner, focusing on query criteria to analyze payments. In summary, the task should:
- **Criteria 1:** Be clear and concise, using minimal jargon.
- **Criteria 2:** Focus on filtering criteria relevant to auditing payments.
- **Criteria 3:** Provide the summary in just two or three sentences.

In [ ]:
# define the task for the Senior Auditor agent
analyze_guideline = Task(
    description=dedent("""\
        Summarize the given guideline and extract its main gist in a very concise and short manner for an IT auditor.

        Utilize the provided guideline text and insights to complete your task effectively.

        Your summary should be:
        1. Clear and concise, avoiding complex jargon to ensure readability.
        2. Focused on the key points relevant to deriving a SQL query for auditing payments.
        3. Explicitly state the filtering criteria that the IT auditor should use when analyzing the payments to identify potential violations of the guideline.
        4. Short, encompassing a total of two to three sentences (this is very important).
    """),
    expected_output=dedent("""\
        A concise and clear summary of the guideline, tailored for an IT auditor, including explicit filtering criteria for analyzing payments.
    """),
    agent=senior_auditor, # assign the task to the senior auditor agent
    max_attempts=1,
    output_file='task_01.txt'
)

In [ ]:
analyze_guideline.execute()

## 5. Establishing the Senior IT Auditor Agent

In this section, we will create the second agent of the audit crew. The **Senior IT Auditor** specializes in querying databases and ensuring that all payments comply with the organization's regulations. Let's walk through the steps to create and assign tasks to this agent.

<img align='center' style="max-width: 850px" src='./senior_it_auditor_agent.png'>

**Figure 4: Senior IT Auditor Agent - Tools and Capabilities**

Before we define the **Senior IT Auditor** agent, we need to set up an in-memory SQLite database that contains sample payment records. This will allow the agent to run SQL queries on these records to check for potential violations of the organizational guidelines.

In [ ]:
# define the metadata object
metadata_obj = MetaData()

# Define the schema for a payments table
payments = Table(
    "payments",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("text", String(4), nullable=False),
    Column("amount", Float, nullable=False),
    Column("date", Date, nullable=False),
)

# insert sample data for payments
payment_data = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 1029, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 1225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 1798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

# create the in-memory SQLite engine and apply the schema
engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

# insert data into the payments table
with engine.begin() as conn:
    for payment in payment_data:
        stmt = insert(payments).values(
            id=payment[0],
            text=payment[1],
            amount=payment[2],
            date=payment[3]
        )
        conn.execute(stmt)

Now that we have set up the in-memory payments database, we need to define a function that allows the **Senior IT Auditor** to run SQL queries on the database. This function will execute the queries and return the results in a structured format.

In [ ]:
# define a function to execute a SQL query on the database
def run_query(query):
    try:
        # execute the query and fetch results
        result = pd.read_sql(query, con=engine)
        
        if result.empty:
            return "No results found."
        
        # format and return the result as a table
        table = tb.tabulate(result, headers=result.columns, tablefmt="grid")
        
        return table
    
    except Exception as e:
        return str(e)

The **Senior IT Auditor** will use the `QueryDatabase` tool to run SQL queries on the payment records. This tool takes a query as input, executes it on the database, and returns the results.

In [ ]:
# define the QueryDatabase tool
class QueryDatabase(BaseTool):

    name: str = "Query database"
    description: str = "Execute SQL database query and return the query result."
    result_as_answer: bool = True

    # define database query
    def _run(self, query: str) -> str:

        try:
            query = query.strip().replace('"', '')

            # echoing cleaned query
            print(f"\nExecuting cleaned query: {query}")

            if not query:
                return "No SQL query provided."

            result = run_query(query)

            return result

        except Exception as exception:
            return str(exception)

In [ ]:
# instantiate the QueryDatabase tool
qdb = QueryDatabase()

Next, we define the **Senior IT Auditor** agent. This agent is responsible for converting the organizational guideline summary into SQL queries to determine potential guideline violations in the payments database. The agent will use the `QueryDatabase` tool to execute these queries.

The Senior IT Auditor agent has the following characteristics:
- **Goal:** Convert guideline summaries into SQL queries for auditing payment records.
- **Backstory:** A highly skilled IT auditor known for analyzing complex systems and ensuring compliance.

In [ ]:
# define the Senior IT Auditor agent
senior_it_auditor = Agent(
    role='Senior IT Auditor',
    goal=dedent("""\
        Convert an organizational guidelines summary into a sql query to determine guideline violations in the database of payment records.
    """),
    backstory=dedent("""\
        You are a highly skilled IT auditor at a leading financial auditing firm. 
        Your expertise includes identifying system vulnerabilities, ensuring regulatory compliance, and optimizing payment processes.
        You excel in analyzing complex IT systems and providing actionable recommendations to enhance security and efficiency.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm, # the agent will use the LLaMA3 model
    tools=[qdb]  # the agent will use the QueryDatabase tool
)

Now that we have defined the **Senior IT Auditor** agent and the tools, we can assign a task to the agent. The task is to translate the filtering criteria provided by the Senior Auditor into an SQL query and execute it on the payments database. The task will retrieve high-value payments that might indicate potential violations.

The task will:

- Use the filtering criteria from the Senior Auditor to construct the SQL query.
- Execute the query and return the results in a structured format.
- Ensure that the task completes with the message ">completed<" once the query has been successfully executed.

In [ ]:
# define the Query Database task
query_database = Task(
    description=dedent("""\
        Translate the filtering criteria provided by the Senior Auditor into a SQL query and execute it to audit the relevant databases for payments exceeding a certain threshold.

        Your actions should encompass:
        
        1. Use the explicit filtering criteria provided by the Senior Auditor to derive an SQL query.
        2. Ensure the query retrieves all relevant fields using * instead of individual field names.
        3. Execute the query and return the results in a structured format (e.g., JSON or a nicely formatted table).

        Your action input should always follow the following format (this is very important):

        {"query": "SELECT * FROM payments WHERE <filtering_criteria>"}

        No quotation marks around the SQL statements. 

        The objective is to identify all high-value payments, which may indicate potential issues requiring further investigation. 
        The task is completed once the query returns ">completed<" at the end.
    """),
    expected_output=dedent("""\
        A table containing all fields (id, text, amount, date) for payments that meet the filtering criteria provided by the Senior Auditor.
    """),
    agent=senior_it_auditor, # assign the task to the senior it auditor agent
    max_attempts=1,
    output_file='task_02.txt'
)

## 6. Establishing the Audit Manager Agent

In this section, we will create the third agent of the audit crew. The **Audit Manager** specializes in turning complex audit findings into concise and accessible reports that can be easily understood by non-technical audiences.

<img align='center' style='max-width: 900px' src='./audit_manager_agent.png'>

**Figure 5: Senior Audit Manager Agent - Tools and Capabilities**

The **Audit Manager** agent will take the findings from the Senior IT Auditor and use them to produce a comprehensive report. The agent’s report will include details of the audit process, findings, and actionable recommendations. 

The **Audit Manager** has the following characteristics:
- **Goal:** To create insightful and accessible audit reports based on audit findings.
- **Backstory:** A celebrated Audit Manager known for producing impactful reports that distill complex audit data into clear insights.

In [ ]:
audit_manager = Agent(
    role='Manager Auditor',
    goal=dedent("""\
        Create comprehensive and compelling audit reports that provide clear insights and actionable recommendations.
    """),
    backstory=dedent("""\
        You are a renowned Audit Manager, celebrated for your ability to produce insightful and impactful audit reports.
        Your reports distill complex audit findings into clear, accessible, and actionable insights, making them valuable to stakeholders.
    """),
    verbose=True,
    allow_delegation=False,
    llm=llm, # the agent will use the LLaMA3 model
    tools=[]  # the Audit Manager doesn't need any tools here
)

Now that we have defined the **Audit Manager** agent, we assign a task to the agent to write the final audit report. The report will be based on the query results from the **Senior IT Auditor** and should be structured to provide clear insights and actionable recommendations to stakeholders.

The task will:

- Be informative and accessible to a non-technical audience.
- Include details about the audit procedures, findings, and recommendations.
- Summarize the key points in a clear and concise manner.

In [ ]:
# Define the report writing task for the Audit Manager
write_report = Task(
    description=dedent("""\
        Using the query results provided by the Senior IT Auditor, write an engaging and comprehensive audit report.

        Your report should be:

        1. Informative and accessible to a non-technical audience.
        2. Clear and concise, avoiding complex jargon to ensure readability.
        3. Comprehensive, covering the audit procedure and associated audit findings.

        Your report should include:

        1. A brief introduction that provides context for the audit.
        2. A detailed description of the performed audit procedures.
        3. A thorough presentation of the obtained audit findings, including payment details.
        4. Actionable recommendations based on the findings.
        5. A final conclusion that summarizes the key points and implications.

        The goal is to create a report that effectively communicates the audit findings and provides valuable insights to stakeholders.
        
        As the report writing expert, you are responsible for producing this report without requesting additional information. 
        Utilize the provided data and insights to complete your task effectively.
    """),
    expected_output=dedent("""\
        A full audit report presented in a clear and accessible manner.
    """),
    agent=audit_manager,  # assign the task to the audit manager agent
    max_attempts=1,
    output_file='task_03.txt'
)

## 7. Establishing the Audit Team

In this final section, we bring together all the agents we have created—the **Senior Auditor**, **Senior IT Auditor**, and **Audit Manager**—and assign them their respective tasks. Together, they form the **Audit Team**, which will collaboratively perform the full audit process.

The **Audit Team** will work sequentially:
1. The **Senior Auditor** will analyze the guidelines and extract key points.
2. The **Senior IT Auditor** will translate those points into SQL queries and run them on the payments database.
3. The **Audit Manager** will compile the findings into a comprehensive audit report.

Let’s get the team to work!

In [ ]:
# instantiate the audit team
audit_team = Crew(
    agents=[senior_auditor, senior_it_auditor, audit_manager],  # assign agents to the audit team
    tasks=[analyze_guideline, query_database, write_report],     # assign tasks to the audit team
    verbose=False,  # set logging level for output visibility
    process=Process.sequential  # set the process to sequential to ensure proper order of execution
)

Now that the **Audit Team** is instantiated, we can run the crew to perform the audit. The agents will work through their tasks sequentially, ensuring each step is completed in sequential order.

In [ ]:
# get the audit team to work
result = audit_team.kickoff()